In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132242")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E745UVS8F to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-132242
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-132242


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cup_cluster_name = 'cpucluster2'

# verify that your cluster does not exist
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({"solver":['newton-cg', 'lbfgs'],
     "penalty":['none', 'l1', 'l2', 'elasticnet'],
     "c": [100, 10, 1.0, 0.1, 0.01]})### YOUR CODE HERE ###

# Specify a Policy
policy =  BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
experiment_folder = r'C:\Users\demouser\Downloads\nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files-master\nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files-master'
est = SKLearn(source_directory=experiment_folder, entry_script='train.py', compute_target='cpu_cluster2', use_docker=False)
      

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='accuracy',  
                                     max_concurrent_runs=None, max_duration_minutes=10080, 
                                     policy=policy, estimator=est, run_config=None, resume_from=None
                                     )

TypeError: __init__() missing 2 required positional arguments: 'primary_metric_goal' and 'max_total_runs'

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
# Create an experiment
experiment_name = 'udacity-project1-estimator-experiment'
experiment = Experiment(workspace = ws, name = experiment_name)
# Run the experiment based on the estimator
run = experiment.submit(config=hyperdrive_config)
# Get Run Details
RunDetails(run).show()
# Wait to complete the experiment. In the Azure Portal we will find the experiment state as preparing --> finished.
run.wait_for_completion(show_output=True)


In [ ]:
import joblib
# Get your best run and save the model from that run.
# Get your best run
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

# save your best model
joblib.dump(value=best_run, filename="udacity_pr.pkl")
### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
#azureml.data.dataset_factory.TabularDatasetFactory.from_delimited_files

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
#from_delimited_files(path, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)
#ds.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run
ws = Workspace.from_config()
experiment2 = Experiment(ws, "udacity-project1-automl")
run2 = experiment2.submit(automl_config, show_output=True)
### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.
import joblib
# Get your best run and save the model from that run.
# Get your best run
best_run2 = run2.get_best_run_by_primary_metric()
best_run_metrics2 = best_run2.get_metrics()
parameter_values2 = best_run2.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run2.id)
print('\n Accuracy:', best_run_metrics2['accuracy'])
print('\n learning rate:',parameter_values2[3])
print('\n keep probability:',parameter_values2[5])
print('\n batch size:',parameter_values2[7])

# save your best model
joblib.dump(value=best_run2, filename="udacity_automl_pr.pkl")
### YOUR CODE HERE ###